In [1]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import registry
from sqlalchemy.orm import relationship

In [2]:
mapper_registry = registry()

In [3]:
@mapper_registry.mapped
class User:
    __tablename__ = "user_account"

    id = Column(Integer, primary_key=True)
    username = Column(String)
    fullname = Column(String)

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return "<User(%r, %r)>" % (self.username, self.fullname)

In [4]:
from sqlalchemy import ForeignKey

@mapper_registry.mapped
class Address:
    __tablename__ = "email_address"

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(ForeignKey("user_account.id"), nullable=False)

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(%r)>" % self.email_address

In [5]:
from sqlalchemy import create_engine

engine = create_engine("sqlite://", echo=True)
with engine.begin() as connection:
    mapper_registry.metadata.create_all(connection)

2023-06-25 10:53:33,590 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 10:53:33,590 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-25 10:53:33,591 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-25 10:53:33,591 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-06-25 10:53:33,592 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-25 10:53:33,592 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("email_address")
2023-06-25 10:53:33,593 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-25 10:53:33,593 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("email_address")
2023-06-25 10:53:33,593 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-25 10:53:33,595 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	username VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-06-25 10:53:33,595 INFO sqlalchemy.engine.Engine [no key 0.00031s] ()
2023-06-25 10:53:33,596 INFO sqlalchemy.engi

In [6]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine, future=True)

with Session.begin() as session:
    session.add_all(
        [
            User(username="spongebob", fullname="Spongebob Squarepants"),
            User(username="sandy", fullname="Sandy Cheeks"),
            User(username="patrick", fullname="Patrick Star"),
        ]
    )

2023-06-25 10:53:33,605 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 10:53:33,607 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-25 10:53:33,607 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ('spongebob', 'Spongebob Squarepants')
2023-06-25 10:53:33,608 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-25 10:53:33,608 INFO sqlalchemy.engine.Engine [cached since 0.001854s ago] ('sandy', 'Sandy Cheeks')
2023-06-25 10:53:33,609 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-25 10:53:33,609 INFO sqlalchemy.engine.Engine [cached since 0.002773s ago] ('patrick', 'Patrick Star')
2023-06-25 10:53:33,610 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
squidward = User(username="squidward", fullname="Squidward Tentacles")
squidward.addresses

[]

In [8]:
# populate this collection with new Address objects.
squidward.addresses = [
    Address(email_address="squidward@gmail.com"),
    Address(email_address="s25@yahoo.com"),
    Address(email_address="squidward@hotmail.com"),
]

print(squidward.addresses[1])

# "back populates" sets up Address.user for each User.address.
print(squidward.addresses[1].user)

<Address('s25@yahoo.com')>
<User('squidward', 'Squidward Tentacles')>


In [9]:
session = Session()

session.add(squidward)
session.new

IdentitySet([<User('squidward', 'Squidward Tentacles')>, <Address('squidward@gmail.com')>, <Address('s25@yahoo.com')>, <Address('squidward@hotmail.com')>])

In [10]:
session.commit()

2023-06-25 10:53:33,646 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 10:53:33,647 INFO sqlalchemy.engine.Engine INSERT INTO user_account (username, fullname) VALUES (?, ?)
2023-06-25 10:53:33,648 INFO sqlalchemy.engine.Engine [cached since 0.04109s ago] ('squidward', 'Squidward Tentacles')
2023-06-25 10:53:33,649 INFO sqlalchemy.engine.Engine INSERT INTO email_address (email_address, user_id) VALUES (?, ?)
2023-06-25 10:53:33,649 INFO sqlalchemy.engine.Engine [generated in 0.00046s] ('squidward@gmail.com', 4)
2023-06-25 10:53:33,650 INFO sqlalchemy.engine.Engine INSERT INTO email_address (email_address, user_id) VALUES (?, ?)
2023-06-25 10:53:33,650 INFO sqlalchemy.engine.Engine [cached since 0.001573s ago] ('s25@yahoo.com', 4)
2023-06-25 10:53:33,651 INFO sqlalchemy.engine.Engine INSERT INTO email_address (email_address, user_id) VALUES (?, ?)
2023-06-25 10:53:33,651 INFO sqlalchemy.engine.Engine [cached since 0.002454s ago] ('squidward@hotmail.com', 4)
2023-06-25 10:53:3

In [11]:
# After the data is expired, first access will be a lazy loading.
squidward.addresses

2023-06-25 10:53:33,657 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 10:53:33,659 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.username AS user_account_username, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2023-06-25 10:53:33,660 INFO sqlalchemy.engine.Engine [generated in 0.00055s] (4,)
2023-06-25 10:53:33,661 INFO sqlalchemy.engine.Engine SELECT email_address.id AS email_address_id, email_address.email_address AS email_address_email_address, email_address.user_id AS email_address_user_id 
FROM email_address 
WHERE ? = email_address.user_id
2023-06-25 10:53:33,662 INFO sqlalchemy.engine.Engine [generated in 0.00039s] (4,)


[<Address('squidward@gmail.com')>,
 <Address('s25@yahoo.com')>,
 <Address('squidward@hotmail.com')>]

In [12]:
# The data stays in the memory until the transaction ends.
squidward.addresses

[<Address('squidward@gmail.com')>,
 <Address('s25@yahoo.com')>,
 <Address('squidward@hotmail.com')>]

In [13]:
# Updating foreign keys.
from sqlalchemy import select

spongebob = session.execute(
    select(User).filter_by(username="spongebob")
).scalar_one()

print(spongebob.addresses)

squidward.addresses[1].user = spongebob
print(spongebob.addresses)
print(squidward.addresses)

2023-06-25 10:53:33,679 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account 
WHERE user_account.username = ?
2023-06-25 10:53:33,679 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('spongebob',)
2023-06-25 10:53:33,681 INFO sqlalchemy.engine.Engine SELECT email_address.id AS email_address_id, email_address.email_address AS email_address_email_address, email_address.user_id AS email_address_user_id 
FROM email_address 
WHERE ? = email_address.user_id
2023-06-25 10:53:33,681 INFO sqlalchemy.engine.Engine [cached since 0.01987s ago] (1,)
[]
[<Address('s25@yahoo.com')>]
[<Address('squidward@gmail.com')>, <Address('squidward@hotmail.com')>]


In [14]:
session.commit()

2023-06-25 10:54:02,269 INFO sqlalchemy.engine.Engine UPDATE email_address SET user_id=? WHERE email_address.id = ?
2023-06-25 10:54:02,270 INFO sqlalchemy.engine.Engine [generated in 0.00093s] (1, 2)
2023-06-25 10:54:02,271 INFO sqlalchemy.engine.Engine COMMIT


In [15]:
# Query multiple tables.
stmt = select(User, Address).where(User.id == Address.user_id)

for row in session.execute(stmt):
    print(row)

2023-06-25 11:04:51,378 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 11:04:51,379 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname, email_address.id AS id_1, email_address.email_address, email_address.user_id 
FROM user_account, email_address 
WHERE user_account.id = email_address.user_id
2023-06-25 11:04:51,380 INFO sqlalchemy.engine.Engine [generated in 0.00048s] ()
(<User('squidward', 'Squidward Tentacles')>, <Address('squidward@gmail.com')>)
(<User('spongebob', 'Spongebob Squarepants')>, <Address('s25@yahoo.com')>)
(<User('squidward', 'Squidward Tentacles')>, <Address('squidward@hotmail.com')>)


In [16]:
stmt = select(User, Address).join(Address)
session.execute(stmt).all()

2023-06-25 11:05:35,541 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname, email_address.id AS id_1, email_address.email_address, email_address.user_id 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id
2023-06-25 11:05:35,542 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ()


[(<User('squidward', 'Squidward Tentacles')>, <Address('squidward@gmail.com')>),
 (<User('spongebob', 'Spongebob Squarepants')>, <Address('s25@yahoo.com')>),
 (<User('squidward', 'Squidward Tentacles')>, <Address('squidward@hotmail.com')>)]

In [17]:
# Being explicit in ON clause.
stmt = select(User, Address).join(Address, User.id == Address.user_id)
session.execute(stmt).all()

2023-06-25 11:10:28,304 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname, email_address.id AS id_1, email_address.email_address, email_address.user_id 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id
2023-06-25 11:10:28,305 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ()


[(<User('squidward', 'Squidward Tentacles')>, <Address('squidward@gmail.com')>),
 (<User('spongebob', 'Spongebob Squarepants')>, <Address('s25@yahoo.com')>),
 (<User('squidward', 'Squidward Tentacles')>, <Address('squidward@hotmail.com')>)]

In [18]:
stmt = (
    select(User, Address)
    .join(User.addresses)
    .where(Address.email_address == "squidward@gmail.com")
)
session.execute(stmt).first()

2023-06-25 11:13:58,959 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname, email_address.id AS id_1, email_address.email_address, email_address.user_id 
FROM user_account JOIN email_address ON user_account.id = email_address.user_id 
WHERE email_address.email_address = ?
2023-06-25 11:13:58,960 INFO sqlalchemy.engine.Engine [generated in 0.00085s] ('squidward@gmail.com',)


(<User('squidward', 'Squidward Tentacles')>, <Address('squidward@gmail.com')>)

In [19]:
# Alias simlar to Core usage.
from sqlalchemy.orm import aliased

a1, a2 = aliased(Address), aliased(Address)

stmt = (
    select(User)
    .join_from(User, a1)
    .join_from(User, a2)
    .where(a1.email_address == "squidward@gmail.com")
    .where(a2.email_address == "squidward@hotmail.com")
)

session.execute(stmt).all()

2023-06-25 11:20:09,339 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account JOIN email_address AS email_address_1 ON user_account.id = email_address_1.user_id JOIN email_address AS email_address_2 ON user_account.id = email_address_2.user_id 
WHERE email_address_1.email_address = ? AND email_address_2.email_address = ?
2023-06-25 11:20:09,340 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ('squidward@gmail.com', 'squidward@hotmail.com')


[(<User('squidward', 'Squidward Tentacles')>,)]

In [21]:
# Subquery similar to Core usage.
from sqlalchemy import func

subq = (
    select(func.count(Address.id).label("count"), Address.user_id)
    .group_by(Address.user_id)
    .subquery()
)

stmt = (
    select(User.username, func.coalesce(subq.c.count, 0))
    .outerjoin(subq, User.id == subq.c.user_id)
)

session.execute(stmt).all()

2023-06-25 11:27:12,720 INFO sqlalchemy.engine.Engine SELECT user_account.username, coalesce(anon_1.count, ?) AS coalesce_1 
FROM user_account LEFT OUTER JOIN (SELECT count(email_address.id) AS count, email_address.user_id AS user_id 
FROM email_address GROUP BY email_address.user_id) AS anon_1 ON user_account.id = anon_1.user_id
2023-06-25 11:27:12,721 INFO sqlalchemy.engine.Engine [cached since 14.6s ago] (0,)


[('spongebob', 1), ('sandy', 0), ('patrick', 0), ('squidward', 2)]

In [22]:
# Eager loading.
with Session() as session:
    for user in session.execute(
        select(User)
    ).scalars():
        print(user, user.addresses)

2023-06-25 11:39:12,530 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 11:39:12,531 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account
2023-06-25 11:39:12,532 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ()
2023-06-25 11:39:12,533 INFO sqlalchemy.engine.Engine SELECT email_address.id AS email_address_id, email_address.email_address AS email_address_email_address, email_address.user_id AS email_address_user_id 
FROM email_address 
WHERE ? = email_address.user_id
2023-06-25 11:39:12,533 INFO sqlalchemy.engine.Engine [cached since 2739s ago] (1,)
<User('spongebob', 'Spongebob Squarepants')> [<Address('s25@yahoo.com')>]
2023-06-25 11:39:12,534 INFO sqlalchemy.engine.Engine SELECT email_address.id AS email_address_id, email_address.email_address AS email_address_email_address, email_address.user_id AS email_address_user_id 
FROM email_address 
WHERE ? = email_address.user_id
2023-06-25 11:39:12,534 INFO s

In [23]:
from sqlalchemy.orm import selectinload

with Session() as session:
    for user in session.execute(
        select(User)
        .options(
            selectinload(User.addresses)
        )
    ).scalars():
        print(user, user.addresses)

2023-06-25 11:48:59,111 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 11:48:59,113 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname 
FROM user_account
2023-06-25 11:48:59,113 INFO sqlalchemy.engine.Engine [generated in 0.00047s] ()
2023-06-25 11:48:59,115 INFO sqlalchemy.engine.Engine SELECT email_address.user_id AS email_address_user_id, email_address.id AS email_address_id, email_address.email_address AS email_address_email_address 
FROM email_address 
WHERE email_address.user_id IN (?, ?, ?, ?)
2023-06-25 11:48:59,116 INFO sqlalchemy.engine.Engine [generated in 0.00040s] (1, 2, 3, 4)
<User('spongebob', 'Spongebob Squarepants')> [<Address('s25@yahoo.com')>]
<User('sandy', 'Sandy Cheeks')> []
<User('patrick', 'Patrick Star')> []
<User('squidward', 'Squidward Tentacles')> [<Address('squidward@gmail.com')>, <Address('squidward@hotmail.com')>]
2023-06-25 11:48:59,117 INFO sqlalchemy.engine.Engine ROLLBACK


In [25]:
from sqlalchemy.orm import joinedload

with Session() as session:
    for address_obj in session.execute(
        select(Address)
        .options(
            joinedload(Address.user, innerjoin=True)
        )
    ).scalars():
        print(address_obj.email_address, address_obj.user.username)

2023-06-25 11:58:41,276 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 11:58:41,278 INFO sqlalchemy.engine.Engine SELECT email_address.id, email_address.email_address, email_address.user_id, user_account_1.id AS id_1, user_account_1.username, user_account_1.fullname 
FROM email_address JOIN user_account AS user_account_1 ON user_account_1.id = email_address.user_id
2023-06-25 11:58:41,278 INFO sqlalchemy.engine.Engine [generated in 0.00043s] ()
squidward@gmail.com squidward
s25@yahoo.com spongebob
squidward@hotmail.com squidward
2023-06-25 11:58:41,279 INFO sqlalchemy.engine.Engine ROLLBACK


In [26]:
# Using joined load together with join.
with Session() as session:
    for address in session.execute(
        select(Address)
        .join(Address.user)
        .where(User.username == "squidward")
        .options(joinedload(Address.user))
    ).scalars():
        print(address, address.user)

2023-06-25 12:10:51,169 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 12:10:51,172 INFO sqlalchemy.engine.Engine SELECT email_address.id, email_address.email_address, email_address.user_id, user_account_1.id AS id_1, user_account_1.username, user_account_1.fullname 
FROM email_address JOIN user_account ON user_account.id = email_address.user_id LEFT OUTER JOIN user_account AS user_account_1 ON user_account_1.id = email_address.user_id 
WHERE user_account.username = ?
2023-06-25 12:10:51,173 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ('squidward',)
<Address('squidward@gmail.com')> <User('squidward', 'Squidward Tentacles')>
<Address('squidward@hotmail.com')> <User('squidward', 'Squidward Tentacles')>
2023-06-25 12:10:51,174 INFO sqlalchemy.engine.Engine ROLLBACK


In [27]:
# Avoid multiple joins.
from sqlalchemy.orm import contains_eager

with Session() as session:
    for address in session.execute(
        select(Address)
        .join(Address.user)
        .where(User.username == "squidward")
        .options(contains_eager(Address.user))
    ).scalars():
        print(address, address.user)

2023-06-25 12:24:14,799 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-25 12:24:14,801 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.username, user_account.fullname, email_address.id AS id_1, email_address.email_address, email_address.user_id 
FROM email_address JOIN user_account ON user_account.id = email_address.user_id 
WHERE user_account.username = ?
2023-06-25 12:24:14,802 INFO sqlalchemy.engine.Engine [generated in 0.00058s] ('squidward',)
<Address('squidward@gmail.com')> <User('squidward', 'Squidward Tentacles')>
<Address('squidward@hotmail.com')> <User('squidward', 'Squidward Tentacles')>
2023-06-25 12:24:14,803 INFO sqlalchemy.engine.Engine ROLLBACK
